# ICV - ACTION INSPECTION

In [ ]:
########################
# load libraries
library(tidyverse)   
library(data.table)  
library(ggplot2)
library(RColorBrewer)
library(colorRamps)
library(repr)

########################
### MULTIPLOT. Source: http://www.cookbook-r.com/Graphs/Multiple_graphs_on_one_page_(ggplot2)/
multiplot <- function(..., plotlist=NULL, file, cols=1, layout=NULL) {
  library(grid)

  # Make a list from the ... arguments and plotlist
  plots <- c(list(...), plotlist)

  numPlots = length(plots)

  # If layout is NULL, then use 'cols' to determine layout
  if (is.null(layout)) {
    # Make the panel
    # ncol: Number of columns of plots
    # nrow: Number of rows needed, calculated from # of cols
    layout <- matrix(seq(1, cols * ceiling(numPlots/cols)),
                    ncol = cols, nrow = ceiling(numPlots/cols))
  }

 if (numPlots==1) {
    print(plots[[1]])

  } else {
    # Set up the page
    grid.newpage()
    pushViewport(viewport(layout = grid.layout(nrow(layout), ncol(layout))))

    # Make each plot, in the correct location
    for (i in 1:numPlots) {
      # Get the i,j matrix positions of the regions that contain this subplot
      matchidx <- as.data.frame(which(layout == i, arr.ind = TRUE))

      print(plots[[i]], vp = viewport(layout.pos.row = matchidx$row,
                                      layout.pos.col = matchidx$col))
    }
  }
}

# Set Working directory

In [ ]:
########################
### SET WORKING DIRECTORY
setwd("/Users/luca/Projects/rl_sepsis/SEPSIS")
#setwd("C:/Users/l.roggeveen/SRL/rl_sepsis/SEPSIS/")
#setwd("D:/ResearchData/rl_sepsis/SEPSIS")

# Run analysis

In [ ]:
# SET OUTPUT PATH
location = getwd()
fig_path = paste(location, '/figures/',sep="")
print(fig_path)

########################
### IMPORT ICV DATA
data_path <- 'data/'
ICV_data<- suppressMessages(suppressWarnings(read_csv(paste0(data_path, 'ICV_data.csv'))))

########################
### IMPORT MIMIC DATA
MIMIC_data_path <- 'MIMIC_data/'
sepsis_df <- suppressMessages(suppressWarnings(read_csv(paste0(MIMIC_data_path, 'sepsis3-df-no-exclusions.csv')) %>% rename(PatientID = icustay_id) %>% select(PatientID, dbsource)))
MIMIC_data <- suppressMessages(suppressWarnings(read_csv(paste0(data_path, 'MIMIC_data.csv'))))

# Combine data with admission source
MIMIC_all <- MIMIC_data %>% left_join(sepsis_df)
summary(as.factor(MIMIC_all$dbsource))

# SUBSET dbsource = METAVISION and show figures
MIMIC_MV = MIMIC_all %>% filter(dbsource == 'metavision') %>% select(-dbsource)

########################
print("done")

In [ ]:
## Expand palette size, ignore warnings about pallet sizes
colourCount = 21 # number of actions
getPalette = suppressWarnings(colorRampPalette(brewer.pal(9, "Set2")))
long_colors = suppressWarnings(colorRampPalette(brewer.pal(12, "Spectral"))(colourCount))

# ICV
icv_plot = ggplot( ICV_data, aes(total_IV, max_VP, color = as.factor(discrete_action))) + 
        geom_point() + scale_color_manual(values = long_colors) + 
        scale_y_log10(limits = c(0.01,10)) + 
        scale_x_log10(limits = c(0.01,10000), labels = scales::comma) +
        ggtitle("Fluids (IV) vs Vasopressor (VP) dose action bins") + 
        theme(legend.position="none") + theme(panel.border = element_blank(), panel.grid.major = element_blank(),
                                              panel.grid.minor = element_blank(), panel.background = element_blank()) +
        labs(subtitle="AmsterdamUMCdb dataset",y = "Max VP Dose (ugram/min)",x = "Total IV fluid (mL)",caption="X and Y Axis on log10 scale")

# MIMIC
mimic_plot = ggplot( MIMIC_MV, aes(total_IV, max_VP, color = as.factor(discrete_action))) + 
        geom_point() + scale_color_manual(values = long_colors) + 
        scale_y_log10(limits = c(0.01,10)) + 
        scale_x_log10(limits = c(0.01,10000), labels = scales::comma) +
        ggtitle("Fluids (IV) vs Vasopressor (VP) dose action bins") + 
        theme(legend.position="none") + theme(panel.border = element_blank(), panel.grid.major = element_blank(), 
                                              panel.grid.minor = element_blank(), panel.background = element_blank()) +
        labs(subtitle="MIMIC dataset",y = "Max VP Dose (ugram/min)",x = "Total IV fluid (mL)",caption="X and Y Axis on log10 scale") 

########################
### PLOT
options(repr.plot.width=12, repr.plot.height=5)
suppressWarnings(multiplot(mimic_plot, icv_plot, cols=2))

########################
### SAFE PNG
options(repr.plot.width=12, repr.plot.height=5)
png(filename = paste(fig_path,"ACTIONS_datasets.png",sep = ""), width = 12, height = 5, units = "in", res = 400, pointsize=6)
suppressWarnings(multiplot(mimic_plot, icv_plot, cols=2))
dev.off()

########################
### SAFE TIFF
options(repr.plot.width=12, repr.plot.height=5)
tiff(filename = paste(fig_path,"Action_dist_datasets.tiff"), width = 12, height = 5, units = "in", res = 200, pointsize=6)
suppressWarnings(multiplot(mimic_plot, icv_plot, cols=2))
dev.off()